# Analysing IXPE Data with gray filter
<hr style="border: 2px solid #fadbac" />

- **Description:** An example on analysing IXPE data with gray filter using heasoftpy.
- **Data:** IXPE observation of X-ray binary **Swift J1727.8-1613** (ObsID 02250901).
- **References:** Veledina et al. 2023 [here](https://iopscience.iop.org/article/10.3847/2041-8213/ad0781) and Ingram et al. 2024 [here](https://iopscience.iop.org/article/10.3847/1538-4357/ad3faf)
- **Requirements:** `heasoftpy`, `xspec`, `matplotlib`

<hr style="border: 2px solid #fadbac" />

## 1. Introduction

This notebook is a tutorial on accessing IXPE data on Sciserver and getting started with analysing them. You will learn to download the data, extract the source and background regions and perform spectro-polarimetric fits.

It also highly recommended that new users read the IXPE Quick Start Guide ([linked here](https://heasarc.gsfc.nasa.gov/docs/ixpe/analysis/IXPE_quickstart.pdf)) and the recommended practices for statistical treatment of IXPE results [here](https://heasarcdev.gsfc.nasa.gov/docs/ixpe/analysis/IXPE_Stats-Advice.pdf).

<div style='color: #333; background: #ffffdf; padding:20px; border: 4px solid #fadbac'>
<b>Running On Sciserver:</b><br>
When running this notebook inside Sciserver, make sure the HEASARC data drive is mounted when initializing the Sciserver compute container. <a href='https://heasarc.gsfc.nasa.gov/docs/sciserver/'>See details here</a>.
<br>
Also, this notebook requires <code>heasoftpy</code>, which is available in the (heasoft) conda environment. You should see (heasoft) at the top right of the notebook. If not, click there and select it.

<b>Running Outside Sciserver:</b><br>
If running outside Sciserver, some changes will be needed, including:<br>
&bull; Make sure heasoftpy and heasoft are installed (<a href='https://heasarc.gsfc.nasa.gov/docs/software/lheasoft/'>Download and Install heasoft</a>).<br>
&bull; Unlike on Sciserver, where the data is available locally, you will need to download the data to your machine.<br>
</div>

## 2. Module Imports
We need the following python modules:

<div style='color: #333; background: #ffffdf; padding:20px; border: 4px solid #fadbac'>
In this example, reprocessing the data is not required. Instead the level 2 data products are sufficient. If you need to reprocess the data, the IXPE tools are available with <code>from heasoftpy import ixpe</code>.
</div>

In [2]:
import glob
import matplotlib.pyplot as plt
import heasoftpy as hsp
from heasoftpy import ixpe
import xspec

## 3. Finding the Data 

On Sciserver, all the HEASARC data ire mounted locally under `/FTP/`, so once we have the path to the data, we can directly access it without the need to download it.

For our exploratory data analysis, we will use an observation of the blazar **Swift J1727.8-1613** (ObsID 02250901)

In [5]:
data_path = "./data/02250901"

Check the contents of this folder

It should contain the standard IXPE data files, which include:
   - `event_l1` and `event_l2`: level 1 and 2 event files, respectively.
   - `auxil`: auxiliary data files, such as exposure maps.
   - `hk`: house-keeping data such as orbit files etc.
    
For a complete description of data formats of the level 1, level 2 and calibration data products, see the support documentation on the [IXPE Website](https://heasarc.gsfc.nasa.gov/docs/ixpe/analysis/#supportdoc)

In [6]:
glob.glob(f'{data_path}/*')

['./data/02250901/event_l2']

## 4. Exploring The Data
To Analyze the data within the notebook, we use `heasoftpy`.

In the folder for each observation, check for a `README` file. This file is included with a description of known issues (if any) with the processing for that observation.

In this *IXPE* example, it is not necessary to reprocess the data. Instead the level 2 data products can be analysed directly. 

In [8]:
# set some input
indir  = data_path
obsid  = indir.split('/')[-1] 

filelist = glob.glob(f'{indir}/event_l2/*.fits')
filelist

['./data/02250901/event_l2/ixpe02250901_det2_evt2_v01.fits',
 './data/02250901/event_l2/ixpe02250901_det1_evt2_v01.fits',
 './data/02250901/event_l2/ixpe02250901_det3_evt2_v01.fits']

We see that there are three files: one event file for each detector. We can examine the structure of these level 2 files.

In [9]:
det2_fits = filelist[0]
det1_fits = filelist[1]
det3_fits = filelist[2]

#print the file structure for event 1 detector file
out = hsp.fstruct(infile=det1_fits).stdout
print(out)

  No. Type     EXTNAME      BITPIX Dimensions(columns)      PCOUNT  GCOUNT
 
   0  PRIMARY                  8     0                           0    1
   1  BINTABLE EVENTS          8     48(10) 1391495              0    1
 
      Column Name                Format     Dims       Units     TLMIN  TLMAX
      1 TRG_ID                     J
      2 TIME                       D                   s
      3 STATUS                     16X
      4 STATUS2                    16X
      5 PI                         J                   chan          0      374
      6 W_MOM                      E
      7 X                          E                   pixel         1      600
      8 Y                          E                   pixel         1      600
      9 Q                          D
     10 U                          D
 
   2  BINTABLE GTI             8     16(2) 29                    0    1
 
      Column Name                Format     Dims       Units     TLMIN  TLMAX
      1 START         

## 5. Extracting the spectro polarimetric data 

### 5.1 Defining the Source and Background Regions

To obtain the source and background spectra from the Level 2 files, we need to define a source region and background region for the extraction. This can also be done using `ds9`. 

For the source, we extract a 80" circle centered on the source.

The region files should be independently defined for each telescope; in this example, the source location has the same celestial coordinates within 0.25" for all three detectors so a single source and a single background region can be used.

No background subtraction is necessary for this source due to its very high count rate.

In [10]:
f = open("src_g.reg", "w")
f.write('circle(17:27:42.6438,-16:12:06.561,80.000")')
f.close()

### 5.2 Running the extractor tools

The `extractor` tool from FTOOLS, can now be used to extract I,Q and U spectra from IXPE Level 2
event lists as shown below. 

The help for the tool can be displayed using the `hsp.extractor?` command. 

First, we extract the source I,Q and U spectra 

In [11]:
#Extract source I,Q and U spectra for DU1
out = hsp.extractor(filename=det1_fits, binlc=10.0, eventsout='NONE', imgfile='NONE',fitsbinlc='NONE', 
              phafile= 'ixpe_det1_src_g_.pha',regionfile='src_g.reg', timefile='NONE', stokes='NEFF', 
              polwcol='W_MOM', tcol='TIME', ecol='PI', xcolf='X', xcolh='X',ycolf='Y', ycolh='Y')
if out.returncode != 0:
    print(out.stdout)
    raise Exception('extractor for det1 failed!')

#Extract source I,Q and U spectra for DU2
out = hsp.extractor(filename=det2_fits, binlc=10.0, eventsout='NONE', imgfile='NONE',fitsbinlc='NONE', 
              phafile= 'ixpe_det2_src_g_.pha',regionfile='src_g.reg', timefile='NONE', stokes='NEFF', 
              polwcol='W_MOM', tcol='TIME', ecol='PI', xcolf='X', xcolh='X',ycolf='Y', ycolh='Y')
if out.returncode != 0:
    print(out.stdout)
    raise Exception('extractor for det2 failed!')

#Extract source I,Q and U spectra for DU3
out = hsp.extractor(filename=det3_fits, binlc=10.0, eventsout='NONE', imgfile='NONE',fitsbinlc='NONE', 
              phafile= 'ixpe_det3_src_g_.pha',regionfile='src_g.reg', timefile='NONE', stokes='NEFF', 
              polwcol='W_MOM', tcol='TIME', ecol='PI', xcolf='X', xcolh='X',ycolf='Y', ycolh='Y')
if out.returncode != 0:
    print(out.stdout)
    raise Exception('extractor for det3 failed!')


### 5.3 Obtaining the Response Files

For the I spectra, you will need to include the RMF (Response Matrix File), and
the ARF (Ancillary Response File). 

For the Q and U spectra, you will need to include the RMF and MRF (Modulation Response File). The MRF is defined by the product of the energy-dependent modulation factor, $\mu$(E) and the ARF.

The location of the calibration files can be obtained through the `hsp.quzcif` tool. Type in `hsp.quzcif?` to get more information on this function. 

Note that the output of the `hsp.quzcif` gives the path to more than one file. This is because there are 3 sets of response files, corresponding to the different weighting schemes.

- For the 'NEFF' weighting, use 'alpha07_`vv`'.
- For the 'SIMPLE' weighting, use 'alpha075simple_`vv`'.
- For the 'UNWEIGHTED' version, use '20170101_`vv`'.

Where `vv` is the version number of the response files. The use of the latest version of the files is recommended.

In following, we use `vv = 02` for the RMF.

In [12]:
# get the on-axis rmf
res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU1',
             filter='-', date='-', time='-',expr='-',codename='MATRIX')

rmf1 = [x.split()[0] for x in res.output if 'alpha075_02'  in x][0]

res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU2',
             filter='-', date='-', time='-',expr='-',codename='MATRIX')

rmf2 = [x.split()[0] for x in res.output if 'alpha075_02'  in x][0]

res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU3',
             filter='-', date='-', time='-',expr='-',codename='MATRIX')

rmf3 = [x.split()[0] for x in res.output if 'alpha075_02'  in x][0]

For the ARF and MRF files, we choose the files specific to the gray filter. For this, we need to specify ```filter='GRAY'``` and note that the file path is preceded by a prefix 'g_'

In [13]:
# get the on-axis arf
res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU1',
             filter='GRAY', date='-', time='-',expr='-',codename='SPECRESP')
arf1 = [x.split()[0] for x in res.output if 'g_alpha075_01'  in x][0]

res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU2',
             filter='GRAY', date='-', time='-',expr='-',codename='SPECRESP')
arf2 = [x.split()[0] for x in res.output if 'g_alpha075_01'  in x][0]

res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU3',
             filter='GRAY', date='-', time='-',expr='-',codename='SPECRESP')
arf3 = [x.split()[0] for x in res.output if 'g_alpha075_01'  in x][0]

In [15]:
# get the on-axis mrf
res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU1',
             filter='GRAY', date='-', time='-',expr='-',codename='MODSPECRESP')
mrf1 = [x.split()[0] for x in res.output if 'g_alpha075_01'  in x][0]

res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU2',
             filter='GRAY', date='-', time='-',expr='-',codename='MODSPECRESP')
mrf2 = [x.split()[0] for x in res.output if 'g_alpha075_01'  in x][0]

res = hsp.quzcif(mission='ixpe', instrument='gpd',detector='DU3',
             filter='GRAY', date='-', time='-',expr='-',codename='MODSPECRESP')
mrf3 = [x.split()[0] for x in res.output if 'g_alpha075_01'  in x][0]

mrf3

'https://heasarc.gsfc.nasa.gov/FTP/caldb/data/ixpe/gpd/cpf/mrf/ixpe_d3_20170101_g_alpha075_01.mrf'

#### 5.31 A note on generating ARF and MRF using _ixpecalcarf_

In this example, we obtain the ARF and MRF files from the HEASARC calibration database, for consistency with the results reported in Veledina et al. 2023. However, we note that the use recently released _ixpecalcarf_ tool is recommended to generate the RMF and MRF files for each observation, as _ixpecalcarf_ correctly accounts for off-axis vignetting and source extraction radius. 

To use this tool, the spacecraft attitude file, part of the engineering housekeeping file suite retrievable from the “hk” subdirectory in the data archive is required to run this. The resulting Response File is appropriate for an observation in which IXPE has been pointed so that the sky location of the target is at the center of the detector, and a 1.0 source extraction region radius (radius=1.0), and NEFF weighting method (weight=1) have been used to extract the spectrum file. (Other weighting options are weight=0 for UNWEIGHTED and weight=2 for SIMPLE).

For more information, please see the Quick start guide, and the _ixpecalcarf_ documentation page. 

To generate the arf and mrf files suitable to use with a gray filter, the parameter ```detfilter``` must be set to 1 (the default is 0 for the normal filter). e.g: 

```
attfile_d1 = glob.glob(f'{data_path}/hk/ixpe*_det1_att_v*.fits.gz')

ixpe.ixpecalcarf(evtfile=det1_fits , attfile=attfile_d1[0], specfile='none', detfilter=1, radius=1.0, arfout='ixpe_det1_src_g.mrf', weight=1, resptype='mrf', clobber='yes')
```

A similar ixpecalcarf command can be used to generate an ARF for Stokes I spectra by setting resptype=arf.

### 5.4 Load data into PyXSPEC and start fitting 

In [17]:
rmf_list = [rmf1,rmf2,rmf3]
mrf_list = [mrf1,mrf2,mrf3]
arf_list = [arf1,arf2,arf3]
du_list = [1,2,3]

xspec.AllData.clear()

x=0 #factor to get the spectrum numbering right 
for (du, rmf_file, mrf_file, arf_file) in zip(du_list, rmf_list, mrf_list, arf_list):

    #Load the I data
    xspec.AllData("%i:%i ixpe_det%i_src_g_I.pha"%(du, du+x, du))
    s = xspec.AllData(du+x)
    
    # #Load response and background files
    s.response = rmf_file
    s.response.arf = arf_file
    #s.background = 'ixpe_det%i_bkg_I.pha'%du
    
    #Load the Q data
    xspec.AllData("%i:%i ixpe_det%i_src_g_Q.pha"%(du, du+x+1, du))
    s = xspec.AllData(du+x+1)
    
    # #Load response and background files
    s.response = rmf_file
    s.response.arf = mrf_file
    #s.background = 'ixpe_det%i_bkg_Q.pha'%du
    
    #Load the U data
    xspec.AllData("%i:%i ixpe_det%i_src_g_U.pha"%(du, du+x+2, du))
    s = xspec.AllData(du+x+2)
    
    # #Load response and background files
    s.response = rmf_file
    s.response.arf = mrf_file
    #s.background = 'ixpe_det%i_bkg_U.pha'%du
    
    x+=2


1 spectrum  in use
 
Spectral Data File: ixpe_det1_src_g_I.pha  Spectrum 1
Net count rate (cts/s) for Spectrum:1  5.645e+01 +/- 5.462e-02
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-375
  Telescope: IXPE Instrument: GPD  Channel Type: PI
  Exposure Time: 1.892e+04 sec
  Filtering Keys: 
    Stokes: 0
 Using fit statistic: chi
 No response loaded.

***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
New filename ( "none" or "/*" to return to the XSPEC prompt): No such file: .rsp
New filename ( "none" or "/*" to return to the XSPEC prompt): No such file: .rsp
New filename ( "none" or "/*" to return to the XSPEC prompt): No such file: .rsp
Spectrum 1 has no response for source 1


Error: cannot read response file https://heasarc.gsfc.nasa.gov/FTP/caldb/data/ixpe/gpd/cpf/rmf/ixpe_d1_20170101_alpha075_02.rmf
terminated at user request


Exception: Error: No response is assigned to source 1 for spectrum 1

In [ ]:
#Ignore all channels except 2-8keV
xspec.AllData.ignore("0.0-2.0, 8.0-**")

In [ ]:
model = xspec.Model('constant(polconst*powerlaw)')

model.polconst.A = 0.04
model.polconst.psi = 2.2
model.powerlaw.PhoIndex = 1.8
model.powerlaw.norm = 34

In [ ]:
m1 = xspec.AllModels(1)
m2 = xspec.AllModels(2)
m3 = xspec.AllModels(3)

m1.constant.factor = 1.0
m1.constant.factor.frozen = True
m2.constant.factor = 0.9
m3.constant.factor = 0.9

In [ ]:
xspec.AllModels.show()

In [ ]:
xspec.Fit.perform()

#### 5.4.1 Adjusting the fit
As you can see, we are unable to find an acceptable fit when tying the parameters to be identical across detector units. There are likely two effects combining to cause this: This could be due to a known charging effect on the detector gain that depends on the source count rate. Second, the model is over-simplified and so to some extent the gain fit will be fitting real features

Thus, we utilize the gain fit command in XSPEC to fit the energy scale.

In [ ]:
r1 = xspec.AllData(1).response
r1.gain.slope = 0.94
r1.gain.offset =0.05

r4 = xspec.AllData(4).response
r4.gain.slope = 0.94
r4.gain.offset =0.05

r7 = xspec.AllData(7).response
r7.gain.slope = 0.94
r7.gain.offset =0.05

xspec.Fit.perform()

### 5.5 Plotting the results

This is done through `matplotlib`.

In [ ]:
xspec.Plot.area=True
xspec.Plot.xAxis='keV'
xspec.Plot('lda')
yVals=xspec.Plot.y()
yErr = xspec.Plot.yErr()
xVals = xspec.Plot.x()
xErr = xspec.Plot.xErr()
mop = xspec.Plot.model()


fig, ax = plt.subplots(figsize=(10,6))
ax.errorbar(xVals, yVals, xerr=xErr, yerr=yErr, fmt='k.', alpha=0.2)
ax.plot(xVals, mop,'r-')
ax.set_xlabel('Energy (keV)')
ax.set_ylabel(r'counts/cm$^2$/s/keV')
ax.set_xscale("log")
ax.set_yscale("log")

In [ ]:
xspec.Plot.area=True
xspec.Plot.xAxis='keV'
xspec.Plot('polangle')
yVals=xspec.Plot.y()
yErr = [abs(y) for y in xspec.Plot.yErr()]
xVals = xspec.Plot.x()
xErr = xspec.Plot.xErr()
mop = xspec.Plot.model()


fig, ax = plt.subplots(figsize=(10,6))
ax.errorbar(xVals, yVals, xerr=xErr, yerr=yErr, fmt='k.', alpha=0.2)
ax.plot(xVals, mop,'r-')
ax.set_xlabel('Energy (keV)')
ax.set_ylabel(r'Polangle')

## 6. Interpreting the results from XSPEC

There are two parameters of interest in our example. These given by the polarization fraction, A,
and polarization angle, $\psi$. The XSPEC error (or uncertainty) command can be used
to deduce confidence intervals for these parameters, and 2D error contours can be determined using the steppar command using the same steps as outlined in the Quick start guide.

## Additional Resources

Visit the IXPE [GOF Website](https://heasarc.gsfc.nasa.gov/docs/ixpe/analysis/) and the IXPE [Project Website at MSFC](https://ixpe.msfc.nasa.gov/for_scientists/index.html) for more resources.